In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm import tqdm_notebook
import warnings
import multiprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from scipy.optimize import minimize  
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
train1 = pd.read_csv('train.csv')
test1 = pd.read_csv('test.csv')
cols = [c for c in train1.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [ ]:
train2 = train[train['wheezy-copper-turtle-magic'] == i]
test2 = test[test['wheezy-copper-turtle-magic'] == i]



In [15]:
def instant_model(train, test, clf = QuadraticDiscriminantAnalysis(0.5)):
    oof = np.zeros(len(train))
    preds = np.zeros(len(test))
    random = RandomForestClassifier()
    
    for i in tqdm_notebook(range(512)):

        train2 = train[train['wheezy-copper-turtle-magic'] == i]
        test2 = test[test['wheezy-copper-turtle-magic'] == i]
        idx1 = train2.index
        idx2 = test2.index
        
        
        
        data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
        # feature selection
        data2 = VarianceThreshold(threshold = 2).fit_transform(data[cols])
        train3 = pd.DataFrame(data2[:train2.shape[0]], index = idx1)
        train3['target'] = train2['target']
        test3 = pd.DataFrame(data2[train2.shape[0]:], index = idx2)

        # Kfold
        skf = StratifiedKFold(n_splits=11, random_state=42)
        for train_index, test_index in skf.split(train3, train3['target']):
            # clf
            clf = clf
            X_train = train3.iloc[train_index, :].drop(["target"], axis = 1)
            X_test = train3.iloc[test_index, :].drop(["target"], axis = 1)
            y_train = train3.iloc[train_index, :]['target']
            y_test = train3.iloc[test_index, :]['target']
            clf.fit(X_train, y_train)
            
            # output
            train_prob = clf.predict_proba(X_train)[:,1]
            test_prob = clf.predict_proba(X_test)[:,1]
            # train_prob = np.exp(clf.decision_function(X_train))
            # test_prob = np.exp(clf.decision_function(X_test))
            oof[idx1[test_index]] = test_prob

            # bagging
            preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits
            #preds[idx2] += np.exp(clf.decision_function(test3)) / skf.n_splits

            print("Chunk {0} Fold {1}".format(i, roc_auc_score(y_test, test_prob)))

    auc = roc_auc_score(train['target'], oof)
    print(f'AUC: {auc:.5}')
    
    return oof, preds

In [16]:
instant_model(train1, test1)

[0.01244114 0.02250953 0.02078024 0.01462716 0.02354224 0.02120623
 0.02955717 0.02200292 0.01820364 0.01705073 0.0396171  0.0103173
 0.02075853 0.01792574 0.02432793 0.04209276 0.00608573 0.03173024
 0.01249144 0.02384821 0.01234012 0.02034949 0.01340218 0.02222413
 0.00722275 0.03399936 0.01996029 0.04614362 0.02795178 0.02786604
 0.03322717 0.01141077 0.03564511 0.01169966 0.01742545 0.02938596
 0.02393492 0.01993119 0.02555407 0.01991883 0.01417834 0.01113531
 0.02636435 0.00530996 0.02416955 0.0281336 ]
Chunk 0 Fold 0.8933333333333333
[0.02611976 0.02504027 0.02692135 0.01537257 0.02120513 0.02432255
 0.01551649 0.01290595 0.0229784  0.01519907 0.0191776  0.00704012
 0.01439761 0.02021483 0.01189987 0.03768025 0.01817622 0.03697744
 0.02435372 0.00800597 0.02796539 0.03181323 0.0074437  0.02053864
 0.0108349  0.04036385 0.01343555 0.04177618 0.01784318 0.00874146
 0.04702905 0.00588185 0.03965011 0.02208357 0.00845514 0.02071538
 0.0230707  0.04278113 0.02521013 0.01269705 0.01955

Chunk 1 Fold 0.9147727272727273
[0.04032873 0.0209898  0.05905218 0.03476437 0.03338564 0.0357917
 0.0219821  0.02866178 0.02742092 0.01795098 0.02051058 0.02075341
 0.01899553 0.02218908 0.02572878 0.02094451 0.04417239 0.03613788
 0.02503261 0.01586826 0.01749525 0.01644527 0.01058965 0.02609775
 0.03622854 0.04940213 0.00565062 0.09219353 0.05088891 0.02071596
 0.02292402 0.03370978 0.02922284 0.01777454]
Chunk 1 Fold 0.9583333333333334
[0.0318369  0.0067887  0.06927831 0.02419272 0.01929059 0.02589018
 0.0439854  0.032055   0.02420008 0.01638556 0.03948063 0.0200028
 0.02488648 0.0511203  0.02094374 0.0314996  0.03901477 0.02855819
 0.02619791 0.01639492 0.01120067 0.01794449 0.0232346  0.02678738
 0.01546488 0.03189812 0.01612356 0.07879422 0.03250561 0.02952242
 0.02621915 0.03895671 0.02537944 0.03396596]
Chunk 1 Fold 0.9772727272727273
[0.03516364 0.0205975  0.0265826  0.02941499 0.03473109 0.00953928
 0.03542016 0.01744908 0.01947587 0.02090956 0.04138249 0.02504818
 0.0195362

[0.02971897 0.02686771 0.02178192 0.039973   0.02011989 0.02681694
 0.02916007 0.04096979 0.02181387 0.01537596 0.02155598 0.02607465
 0.03983698 0.01640345 0.02426573 0.02074675 0.03143689 0.02718828
 0.03595628 0.0257455  0.01563404 0.01644634 0.01884798 0.02780871
 0.04282717 0.02829268 0.0270654  0.05160322 0.01795112 0.01841149
 0.0257731  0.03003333 0.02536923 0.01421109 0.0181014  0.02478819
 0.05502688]
Chunk 3 Fold 0.9443478260869564
[0.0175794  0.01585455 0.0156798  0.02522198 0.03481818 0.02656628
 0.02782394 0.01933294 0.01583037 0.03508012 0.01857358 0.03238626
 0.02836511 0.03850663 0.01579921 0.02892502 0.01146448 0.01370085
 0.02024897 0.01857862 0.01451684 0.02908785 0.01786935 0.02586549
 0.03803374 0.01345164 0.03635169 0.04118125 0.04267763 0.02619192
 0.02028303 0.02525948 0.03276393 0.03993406 0.0164968  0.0140591
 0.02366397 0.02835225 0.0305371  0.01782478 0.00526182]
Chunk 4 Fold 0.921875
[0.01381046 0.02390904 0.0198047  0.01848132 0.03214677 0.01378623
 0.024

[0.02253603 0.02485396 0.01935883 0.01439315 0.02730417 0.01789803
 0.0312236  0.0355238  0.03086597 0.03276679 0.02500837 0.01168491
 0.0222429  0.03701852 0.05495491 0.02453455 0.02568215 0.01487265
 0.01527164 0.02491533 0.02701607 0.0283046  0.01306224 0.02696435
 0.01264758 0.02138715 0.03597633 0.0282779  0.05296518 0.02695794
 0.01663923 0.01672272 0.05531135 0.02085068 0.01492695 0.00733415
 0.02544423 0.01628145 0.02157173 0.0184479 ]
Chunk 5 Fold 0.9528985507246377
[0.02113528 0.01165786 0.00873697 0.01779732 0.0140948  0.02161192
 0.01853244 0.03762262 0.03951304 0.04209495 0.01414498 0.02576144
 0.02040943 0.01676403 0.03737434 0.01767849 0.04860787 0.01666421
 0.01954335 0.01093094 0.03600655 0.01798341 0.03983629 0.03196956
 0.02526264 0.02152477 0.03744669 0.02610164 0.01694049 0.03312594
 0.01959349 0.01477608 0.06188893 0.01379513 0.03935665 0.0247831
 0.01449519 0.01751056 0.02583088 0.02109569]
Chunk 5 Fold 0.9927536231884058
[0.01851426 0.02280286 0.02453457 0.01894

[0.02413124 0.0437306  0.01462805 0.02640522 0.01135395 0.02645116
 0.02922125 0.0097466  0.02656992 0.02420863 0.0142324  0.01959137
 0.06067282 0.01674121 0.01733215 0.02494573 0.02683054 0.01649097
 0.02249944 0.01436857 0.01402847 0.01544074 0.02063407 0.03614402
 0.03049712 0.02687537 0.02974523 0.01590095 0.04912934 0.01465303
 0.0239412  0.01635581 0.02099025 0.02475199 0.01926917 0.08038713
 0.0256452  0.02166276 0.02671898 0.01707737]
Chunk 7 Fold 0.9092627599243855
[0.01690258 0.02448575 0.02655248 0.02352826 0.01130203 0.03474585
 0.01345262 0.01635159 0.03429383 0.01543505 0.01286736 0.01901888
 0.07918291 0.02862738 0.01952429 0.01931258 0.02075367 0.01232543
 0.03114038 0.01754547 0.02491568 0.01049763 0.02432896 0.02418119
 0.03949052 0.02355225 0.02220582 0.01682769 0.05515211 0.02098773
 0.02734976 0.01504275 0.01732603 0.02877128 0.04202587 0.06509709
 0.01705741 0.01669172 0.02592936 0.00522076]
Chunk 7 Fold 0.9867674858223062
[0.01910965 0.01561226 0.01641987 0.0387

...

[0.03340603 0.01298724 0.06651174 0.03201916 0.01739    0.0234403
 0.01177544 0.02464423 0.02573801 0.01964232 0.01303868 0.02455707
 0.01115567 0.02281298 0.03427498 0.01475017 0.0166038  0.02092977
 0.02366445 0.01843616 0.0082986  0.02427651 0.01421805 0.01794147
 0.00963951 0.01655521 0.02175908 0.02309164 0.01376628 0.01619557
 0.01166649 0.02573438 0.02398737 0.02062913 0.03294439 0.03760654
 0.02347723 0.01775457 0.02540331 0.02373762 0.01599673 0.02386626
 0.04998162 0.03369423]
Chunk 506 Fold 0.9809027777777779
[0.04071254 0.01215895 0.05296584 0.04545435 0.0203835  0.01931063
 0.00676302 0.02470315 0.01490005 0.01273893 0.02772124 0.02109677
 0.00845477 0.02399869 0.02489258 0.01886724 0.02581918 0.01749321
 0.02356681 0.01825235 0.01551813 0.02733668 0.01876295 0.01633111
 0.01872998 0.0409686  0.02001709 0.03218272 0.02427836 0.02158241
 0.01679772 0.01983807 0.03394961 0.0274005  0.01552356 0.0138949
 0.02294211 0.0192917  0.02182183 0.01918632 0.00911701 0.01919334
 0.042

Chunk 508 Fold 0.9774305555555556
[0.01493984 0.01758778 0.01037968 0.03021277 0.02426074 0.02014161
 0.02751805 0.02375673 0.0151965  0.01545159 0.03667312 0.01863396
 0.006772   0.02506022 0.02228501 0.03269679 0.0269172  0.02734311
 0.02096023 0.02359478 0.03938017 0.02081102 0.00963143 0.0349449
 0.01323771 0.01934388 0.02803751 0.01904496 0.03018613 0.01266736
 0.02005532 0.02903561 0.01916686 0.02608321 0.02433255 0.02082128
 0.03548599 0.04803342 0.02230377 0.02462274 0.03207022 0.02366821
 0.00665402]
Chunk 508 Fold 0.935763888888889
[0.02573069 0.02736629 0.01438598 0.03112672 0.0181085  0.01436344
 0.02724195 0.02657134 0.01485378 0.02338569 0.01875319 0.01446806
 0.01655919 0.01930353 0.02092481 0.01940159 0.03637387 0.01844244
 0.02667755 0.01041081 0.05354987 0.02348123 0.0337034  0.01253158
 0.02106883 0.02534068 0.02155955 0.01706477 0.01997459 0.0192929
 0.02506378 0.02681218 0.03322363 0.01935175 0.01874052 0.04212548
 0.03076902 0.03948695 0.02426174 0.01572107 0.0163

[0.0151645  0.01845643 0.02323177 0.02563456 0.01633332 0.01869706
 0.02172126 0.02234088 0.01273137 0.02124003 0.01817862 0.01692912
 0.0258552  0.04132662 0.0300994  0.01789769 0.01587743 0.02111526
 0.03442632 0.01788427 0.0127995  0.0309392  0.0301656  0.02651731
 0.0194774  0.02027054 0.0428679  0.04734862 0.02263415 0.02058296
 0.02133353 0.02219968 0.02533032 0.03873975 0.01883707 0.03338303
 0.0040919  0.01752334 0.0165676  0.02172753 0.02078888 0.02763414
 0.02309893]
Chunk 510 Fold 0.9655797101449276
[0.01280544 0.0143966  0.00820714 0.04627158 0.01877885 0.02357353
 0.0149425  0.02030958 0.01425226 0.01262146 0.01108657 0.02029292
 0.0246135  0.0255556  0.01733071 0.01935568 0.01865258 0.01775784
 0.01557478 0.0361666  0.03486607 0.04605184 0.02041589 0.03959674
 0.01347145 0.03021423 0.01990207 0.02594848 0.01900582 0.02069164
 0.01890043 0.01906867 0.03372704 0.04773413 0.02074795 0.03551468
 0.0165165  0.0103378  0.01981856 0.02103798 0.03103568 0.04219564
 0.020655  ]
Ch

AUC: 0.9649


(array([2.50161506e-07, 3.36159513e-04, 9.95703441e-01, ...,
        1.01700829e-09, 9.96815335e-01, 3.62897734e-05]),
 array([9.99970912e-01, 9.89205441e-01, 2.35631736e-07, ...,
        9.99999998e-01, 9.99990551e-01, 1.27644449e-11]))

In [11]:
instant_model(train1, test1, clf = KNeighborsClassifier(n_neighbors = 7, p = 2, weights = 'distance'))

Chunk 0 Fold 0.9233333333333332
Chunk 0 Fold 0.9208333333333334
Chunk 0 Fold 0.8983333333333333
Chunk 0 Fold 0.9416666666666667
Chunk 0 Fold 0.9816666666666667
Chunk 0 Fold 0.9475000000000001
Chunk 0 Fold 0.84
Chunk 0 Fold 0.96875
Chunk 0 Fold 0.8715277777777778
Chunk 0 Fold 0.9565972222222222
Chunk 0 Fold 0.9384057971014492
Chunk 1 Fold 0.8596014492753623
Chunk 1 Fold 1.0
Chunk 1 Fold 0.947463768115942
Chunk 1 Fold 0.907608695652174
Chunk 1 Fold 0.9734848484848485
Chunk 1 Fold 0.9356060606060606
Chunk 1 Fold 0.9526515151515151
Chunk 1 Fold 0.9299242424242424
Chunk 1 Fold 0.9090909090909091
Chunk 1 Fold 0.9517045454545454
Chunk 1 Fold 0.9356060606060606
Chunk 2 Fold 0.9659735349716446
Chunk 2 Fold 0.9189723320158103
Chunk 2 Fold 0.9466403162055336
Chunk 2 Fold 0.9278656126482214
Chunk 2 Fold 0.9031620553359684
Chunk 2 Fold 0.8774703557312253
Chunk 2 Fold 0.8695652173913044
Chunk 2 Fold 0.9920948616600791
Chunk 2 Fold 0.9110671936758894
Chunk 2 Fold 0.9110671936758894
Chunk 2 Fold 0.952

Chunk 23 Fold 0.8659420289855073
Chunk 23 Fold 0.9710144927536233
Chunk 23 Fold 0.9157608695652174
Chunk 23 Fold 0.9239130434782609
Chunk 23 Fold 0.9365942028985507
Chunk 23 Fold 0.9329710144927537
Chunk 23 Fold 0.9746376811594203
Chunk 24 Fold 0.9375
Chunk 24 Fold 0.9375
Chunk 24 Fold 0.8968253968253967
Chunk 24 Fold 0.9583333333333333
Chunk 24 Fold 0.9523809523809524
Chunk 24 Fold 0.873015873015873
Chunk 24 Fold 0.9603174603174602
Chunk 24 Fold 0.9226190476190476
Chunk 24 Fold 0.9484126984126985
Chunk 24 Fold 0.943064182194617
Chunk 24 Fold 0.8395445134575569
Chunk 25 Fold 0.9081439393939393
Chunk 25 Fold 0.899621212121212
Chunk 25 Fold 0.8849206349206349
Chunk 25 Fold 0.9087301587301587
Chunk 25 Fold 0.9484126984126983
Chunk 25 Fold 0.94824016563147
Chunk 25 Fold 0.9648033126293996
Chunk 25 Fold 0.9440993788819876
Chunk 25 Fold 0.9503105590062112
Chunk 25 Fold 0.8902691511387164
Chunk 25 Fold 0.9730848861283643
Chunk 26 Fold 0.9090909090909091
Chunk 26 Fold 0.9110671936758893
Chunk 

Chunk 46 Fold 0.9338842975206612
Chunk 47 Fold 0.9443478260869566
Chunk 47 Fold 0.8208695652173914
Chunk 47 Fold 0.9356521739130435
Chunk 47 Fold 0.888695652173913
Chunk 47 Fold 0.9600000000000001
Chunk 47 Fold 0.9339130434782609
Chunk 47 Fold 0.8747826086956522
Chunk 47 Fold 0.9109848484848485
Chunk 47 Fold 0.837121212121212
Chunk 47 Fold 0.9848484848484848
Chunk 47 Fold 0.9166666666666666
Chunk 48 Fold 0.9433333333333334
Chunk 48 Fold 0.9141666666666668
Chunk 48 Fold 0.9566666666666666
Chunk 48 Fold 0.9041666666666667
Chunk 48 Fold 0.95
Chunk 48 Fold 0.9460869565217391
Chunk 48 Fold 0.9147826086956522
Chunk 48 Fold 0.8939130434782608
Chunk 48 Fold 0.8731884057971014
Chunk 48 Fold 0.9329710144927537
Chunk 48 Fold 0.9764492753623188
Chunk 49 Fold 0.9655797101449275
Chunk 49 Fold 0.9039855072463768
Chunk 49 Fold 0.9356884057971016
Chunk 49 Fold 0.9094202898550725
Chunk 49 Fold 0.9565217391304348
Chunk 49 Fold 0.9764492753623188
Chunk 49 Fold 0.9329710144927535
Chunk 49 Fold 0.9149338374

Chunk 70 Fold 0.8827272727272727
Chunk 70 Fold 0.9163636363636364
Chunk 70 Fold 0.9927272727272728
Chunk 70 Fold 0.9572727272727273
Chunk 70 Fold 0.8981818181818182
Chunk 70 Fold 0.9563636363636363
Chunk 70 Fold 0.9488636363636364
Chunk 70 Fold 0.9564393939393939
Chunk 71 Fold 0.9199999999999999
Chunk 71 Fold 0.888888888888889
Chunk 71 Fold 0.9836956521739131
Chunk 71 Fold 0.8541666666666666
Chunk 71 Fold 0.967391304347826
Chunk 71 Fold 0.9067028985507247
Chunk 71 Fold 0.9148550724637681
Chunk 71 Fold 0.9855072463768116
Chunk 71 Fold 0.9239130434782609
Chunk 71 Fold 0.9402173913043478
Chunk 71 Fold 0.9384057971014492
Chunk 72 Fold 0.9265873015873016
Chunk 72 Fold 0.9146825396825397
Chunk 72 Fold 0.875
Chunk 72 Fold 0.9503968253968254
Chunk 72 Fold 0.8819444444444444
Chunk 72 Fold 0.8551587301587301
Chunk 72 Fold 0.9365079365079365
Chunk 72 Fold 0.9325396825396826
Chunk 72 Fold 0.9206349206349207
Chunk 72 Fold 0.9464285714285715
Chunk 72 Fold 0.9565217391304348
Chunk 73 Fold 0.975694444

Chunk 93 Fold 0.8063241106719368
Chunk 93 Fold 0.8379446640316206
Chunk 93 Fold 0.9308300395256918
Chunk 93 Fold 0.9802371541501976
Chunk 94 Fold 0.8723776223776225
Chunk 94 Fold 0.958041958041958
Chunk 94 Fold 0.9108391608391609
Chunk 94 Fold 0.958041958041958
Chunk 94 Fold 0.8741258741258742
Chunk 94 Fold 0.9194139194139195
Chunk 94 Fold 0.8219047619047618
Chunk 94 Fold 0.9085714285714285
Chunk 94 Fold 0.9466666666666667
Chunk 94 Fold 0.8761904761904762
Chunk 94 Fold 0.9504761904761905
Chunk 95 Fold 0.9420289855072465
Chunk 95 Fold 0.9601449275362319
Chunk 95 Fold 0.9021739130434783
Chunk 95 Fold 0.948960302457467
Chunk 95 Fold 0.9754253308128545
Chunk 95 Fold 0.9224952741020793
Chunk 95 Fold 0.9054820415879017
Chunk 95 Fold 0.9281663516068053
Chunk 95 Fold 0.8412098298676748
Chunk 95 Fold 0.9367588932806323
Chunk 95 Fold 0.9110671936758894
Chunk 96 Fold 0.9177693761814745
Chunk 96 Fold 0.8629489603024575
Chunk 96 Fold 0.9735349716446124
Chunk 96 Fold 0.8714555765595463
Chunk 96 Fold

...

Chunk 138 Fold 0.9584120982986768
Chunk 138 Fold 0.9206049149338374
Chunk 139 Fold 0.9528985507246377
Chunk 139 Fold 0.9619565217391305
Chunk 139 Fold 0.9420289855072465
Chunk 139 Fold 0.9510869565217391
Chunk 139 Fold 0.9486166007905139
Chunk 139 Fold 0.932806324110672
Chunk 139 Fold 0.8774703557312253
Chunk 139 Fold 0.924901185770751
Chunk 139 Fold 0.9130434782608695
Chunk 139 Fold 0.8804347826086957
Chunk 139 Fold 0.883399209486166
Chunk 140 Fold 0.9275362318840579
Chunk 140 Fold 0.9257246376811594
Chunk 140 Fold 0.9402173913043478
Chunk 140 Fold 0.947463768115942
Chunk 140 Fold 0.947463768115942
Chunk 140 Fold 0.9673913043478259
Chunk 140 Fold 0.9356060606060607
Chunk 140 Fold 0.9640151515151515
Chunk 140 Fold 0.9664031620553359
Chunk 140 Fold 0.9664031620553359
Chunk 140 Fold 0.9466403162055336
Chunk 141 Fold 0.9427083333333333
Chunk 141 Fold 0.8993055555555556
Chunk 141 Fold 0.8993055555555556
Chunk 141 Fold 0.9878472222222222
Chunk 141 Fold 0.9305555555555555
Chunk 141 Fold 0.98

Chunk 161 Fold 0.9384057971014493
Chunk 161 Fold 0.9329710144927537
Chunk 161 Fold 0.9092627599243857
Chunk 161 Fold 0.8790170132325141
Chunk 161 Fold 0.9064272211720227
Chunk 161 Fold 0.9366729678638942
Chunk 162 Fold 0.950850661625709
Chunk 162 Fold 0.8761814744801513
Chunk 162 Fold 0.8827977315689981
Chunk 162 Fold 0.91699604743083
Chunk 162 Fold 0.9209486166007905
Chunk 162 Fold 0.8537549407114624
Chunk 162 Fold 0.8774703557312252
Chunk 162 Fold 0.8745059288537549
Chunk 162 Fold 0.9214876033057852
Chunk 162 Fold 0.9297520661157025
Chunk 162 Fold 0.8853305785123967
Chunk 163 Fold 0.8814229249011858
Chunk 163 Fold 0.9604743083003953
Chunk 163 Fold 0.8675889328063241
Chunk 163 Fold 0.9881422924901185
Chunk 163 Fold 0.9664031620553359
Chunk 163 Fold 0.9169960474308301
Chunk 163 Fold 0.9628099173553719
Chunk 163 Fold 0.9524793388429752
Chunk 163 Fold 0.8842975206611571
Chunk 163 Fold 0.9504132231404959
Chunk 163 Fold 0.9338842975206613
Chunk 164 Fold 0.9582608695652174
Chunk 164 Fold 0.

Chunk 184 Fold 0.9528985507246377
Chunk 184 Fold 0.8586956521739131
Chunk 184 Fold 0.9891304347826088
Chunk 184 Fold 0.9329710144927537
Chunk 184 Fold 0.9456521739130435
Chunk 184 Fold 0.9275362318840581
Chunk 184 Fold 0.9764492753623188
Chunk 184 Fold 0.9565217391304348
Chunk 184 Fold 0.8712121212121212
Chunk 184 Fold 0.9526515151515151
Chunk 184 Fold 0.9356060606060606
Chunk 185 Fold 0.9619047619047619
Chunk 185 Fold 0.9619047619047618
Chunk 185 Fold 0.9571428571428571
Chunk 185 Fold 0.995
Chunk 185 Fold 0.9287500000000001
Chunk 185 Fold 0.9699999999999999
Chunk 185 Fold 0.9125
Chunk 185 Fold 0.9575
Chunk 185 Fold 0.9099999999999999
Chunk 185 Fold 0.9375
Chunk 185 Fold 0.9736842105263158
Chunk 186 Fold 0.9616666666666667
Chunk 186 Fold 0.9616666666666666
Chunk 186 Fold 0.99
Chunk 186 Fold 0.9521739130434782
Chunk 186 Fold 0.9408695652173913
Chunk 186 Fold 0.9565217391304348
Chunk 186 Fold 0.8939130434782608
Chunk 186 Fold 0.9408695652173913
Chunk 186 Fold 0.9495652173913044
Chunk 186

Chunk 206 Fold 0.9319470699432891
Chunk 206 Fold 0.8695652173913042
Chunk 206 Fold 0.9209486166007905
Chunk 206 Fold 0.8705533596837945
Chunk 207 Fold 0.8670634920634921
Chunk 207 Fold 0.886904761904762
Chunk 207 Fold 0.9335317460317459
Chunk 207 Fold 0.9345238095238095
Chunk 207 Fold 0.9316770186335405
Chunk 207 Fold 0.8944099378881988
Chunk 207 Fold 0.9296066252587992
Chunk 207 Fold 0.8757763975155279
Chunk 207 Fold 0.9399585921325052
Chunk 207 Fold 0.9689440993788819
Chunk 207 Fold 0.941304347826087
Chunk 208 Fold 0.9654545454545453
Chunk 208 Fold 0.8636363636363636
Chunk 208 Fold 0.8727272727272728
Chunk 208 Fold 0.9327272727272727
Chunk 208 Fold 0.9363636363636364
Chunk 208 Fold 0.9263636363636364
Chunk 208 Fold 0.9472727272727273
Chunk 208 Fold 0.9272727272727274
Chunk 208 Fold 0.8845454545454546
Chunk 208 Fold 0.9623015873015872
Chunk 208 Fold 0.9047619047619048
Chunk 209 Fold 0.9426877470355731
Chunk 209 Fold 0.9545454545454545
Chunk 209 Fold 0.9723320158102767
Chunk 209 Fold 0

KeyboardInterrupt: 

In [24]:
instant_model(train1, test1, clf = RidgeClassifier(alpha = 1.1))

Chunk 0 Fold 0.905
Chunk 0 Fold 0.785
Chunk 0 Fold 0.8016666666666667
Chunk 0 Fold 0.8116666666666668
Chunk 0 Fold 0.7483333333333333
Chunk 0 Fold 0.84
Chunk 0 Fold 0.7983333333333333
Chunk 0 Fold 0.7430555555555556
Chunk 0 Fold 0.8715277777777778
Chunk 0 Fold 0.861111111111111
Chunk 0 Fold 0.7880434782608696
Chunk 1 Fold 0.7518115942028986
Chunk 1 Fold 0.9293478260869565
Chunk 1 Fold 0.9710144927536232
Chunk 1 Fold 0.8876811594202899
Chunk 1 Fold 0.8503787878787878
Chunk 1 Fold 0.8484848484848484
Chunk 1 Fold 0.9488636363636365
Chunk 1 Fold 0.8806818181818181
Chunk 1 Fold 0.7481060606060606
Chunk 1 Fold 0.9583333333333333
Chunk 1 Fold 0.8693181818181818
Chunk 2 Fold 0.8109640831758034
Chunk 2 Fold 0.7351778656126483
Chunk 2 Fold 0.7588932806324111
Chunk 2 Fold 0.7747035573122529
Chunk 2 Fold 0.774703557312253
Chunk 2 Fold 0.7312252964426877
Chunk 2 Fold 0.6798418972332017
Chunk 2 Fold 0.9110671936758893
Chunk 2 Fold 0.7312252964426879
Chunk 2 Fold 0.7924901185770752
Chunk 2 Fold 0.824

Chunk 23 Fold 0.730072463768116
Chunk 23 Fold 0.6521739130434783
Chunk 23 Fold 0.7282608695652175
Chunk 23 Fold 0.8097826086956522
Chunk 23 Fold 0.7409420289855072
Chunk 23 Fold 0.6829710144927535
Chunk 24 Fold 0.9242424242424243
Chunk 24 Fold 0.9261363636363636
Chunk 24 Fold 0.876984126984127
Chunk 24 Fold 0.9801587301587301
Chunk 24 Fold 0.9047619047619047
Chunk 24 Fold 0.878968253968254
Chunk 24 Fold 0.9226190476190476
Chunk 24 Fold 0.8988095238095238
Chunk 24 Fold 0.9027777777777777
Chunk 24 Fold 0.8964803312629399
Chunk 24 Fold 0.8033126293995859
Chunk 25 Fold 0.7840909090909091
Chunk 25 Fold 0.8087121212121212
Chunk 25 Fold 0.7777777777777778
Chunk 25 Fold 0.7876984126984127
Chunk 25 Fold 0.9027777777777778
Chunk 25 Fold 0.8695652173913043
Chunk 25 Fold 0.8737060041407867
Chunk 25 Fold 0.8695652173913044
Chunk 25 Fold 0.7846790890269151
Chunk 25 Fold 0.8012422360248448
Chunk 25 Fold 0.8923395445134575
Chunk 26 Fold 0.66600790513834
Chunk 26 Fold 0.7964426877470356
Chunk 26 Fold 0

Chunk 48 Fold 0.7583333333333333
Chunk 48 Fold 0.7333333333333333
Chunk 48 Fold 0.8683333333333333
Chunk 48 Fold 0.8783333333333333
Chunk 48 Fold 0.895
Chunk 48 Fold 0.7634782608695652
Chunk 48 Fold 0.7008695652173913
Chunk 48 Fold 0.7547826086956522
Chunk 48 Fold 0.6847826086956522
Chunk 48 Fold 0.7771739130434783
Chunk 48 Fold 0.8804347826086957
Chunk 49 Fold 0.8405797101449275
Chunk 49 Fold 0.7572463768115941
Chunk 49 Fold 0.7844202898550725
Chunk 49 Fold 0.7300724637681159
Chunk 49 Fold 0.8695652173913044
Chunk 49 Fold 0.8152173913043477
Chunk 49 Fold 0.7228260869565217
Chunk 49 Fold 0.8034026465028355
Chunk 49 Fold 0.8204158790170132
Chunk 49 Fold 0.7674858223062382
Chunk 49 Fold 0.7826086956521738
Chunk 50 Fold 0.7369230769230769
Chunk 50 Fold 0.8907692307692308
Chunk 50 Fold 0.7661538461538461
Chunk 50 Fold 0.7153846153846154
Chunk 50 Fold 0.7815384615384616
Chunk 50 Fold 0.6624
Chunk 50 Fold 0.848
Chunk 50 Fold 0.7766666666666667
Chunk 50 Fold 0.7883333333333333
Chunk 50 Fold 0

Chunk 71 Fold 0.911231884057971
Chunk 71 Fold 0.8514492753623188
Chunk 71 Fold 0.8405797101449276
Chunk 71 Fold 0.8079710144927537
Chunk 72 Fold 0.8650793650793651
Chunk 72 Fold 0.8214285714285714
Chunk 72 Fold 0.759920634920635
Chunk 72 Fold 0.8194444444444445
Chunk 72 Fold 0.753968253968254
Chunk 72 Fold 0.8015873015873016
Chunk 72 Fold 0.8412698412698413
Chunk 72 Fold 0.867063492063492
Chunk 72 Fold 0.8353174603174603
Chunk 72 Fold 0.7738095238095238
Chunk 72 Fold 0.8674948240165632
Chunk 73 Fold 0.763888888888889
Chunk 73 Fold 0.7986111111111112
Chunk 73 Fold 0.796875
Chunk 73 Fold 0.8697916666666666
Chunk 73 Fold 0.7447916666666666
Chunk 73 Fold 0.6840277777777778
Chunk 73 Fold 0.8003472222222222
Chunk 73 Fold 0.8576388888888888
Chunk 73 Fold 0.7504725897920604
Chunk 73 Fold 0.7088846880907372
Chunk 73 Fold 0.8620037807183365
Chunk 74 Fold 0.6798418972332017
Chunk 74 Fold 0.6798418972332017
Chunk 74 Fold 0.78099173553719
Chunk 74 Fold 0.7789256198347108
Chunk 74 Fold 0.72520661157

Chunk 95 Fold 0.7971014492753623
Chunk 95 Fold 0.871376811594203
Chunk 95 Fold 0.8496376811594203
Chunk 95 Fold 0.7599243856332702
Chunk 95 Fold 0.827977315689981
Chunk 95 Fold 0.7448015122873346
Chunk 95 Fold 0.7088846880907372
Chunk 95 Fold 0.8638941398865785
Chunk 95 Fold 0.8752362948960303
Chunk 95 Fold 0.8794466403162056
Chunk 95 Fold 0.7707509881422925
Chunk 96 Fold 0.6937618147448014
Chunk 96 Fold 0.8582230623818525
Chunk 96 Fold 0.8468809073724007
Chunk 96 Fold 0.7939508506616257
Chunk 96 Fold 0.7996219281663516
Chunk 96 Fold 0.7051039697542533
Chunk 96 Fold 0.7164461247637051
Chunk 96 Fold 0.6994328922495273
Chunk 96 Fold 0.6389413988657845
Chunk 96 Fold 0.7964426877470355
Chunk 96 Fold 0.7687747035573123
Chunk 97 Fold 0.7234848484848485
Chunk 97 Fold 0.740530303030303
Chunk 97 Fold 0.7784090909090908
Chunk 97 Fold 0.8616600790513834
Chunk 97 Fold 0.7509881422924901
Chunk 97 Fold 0.8557312252964427
Chunk 97 Fold 0.8952569169960474
Chunk 97 Fold 0.8488612836438924
Chunk 97 Fold

...

Chunk 420 Fold 0.8822314049586777
Chunk 420 Fold 0.8939393939393938
Chunk 420 Fold 0.7619047619047619
Chunk 421 Fold 0.730072463768116
Chunk 421 Fold 0.7989130434782609
Chunk 421 Fold 0.8514492753623187
Chunk 421 Fold 0.782608695652174
Chunk 421 Fold 0.8260869565217391
Chunk 421 Fold 0.8409090909090909
Chunk 421 Fold 0.7765151515151515
Chunk 421 Fold 0.6950757575757576
Chunk 421 Fold 0.7154150197628458
Chunk 421 Fold 0.7727272727272727
Chunk 421 Fold 0.7707509881422925
Chunk 422 Fold 0.7169421487603306
Chunk 422 Fold 0.8388429752066116
Chunk 422 Fold 0.8099173553719008
Chunk 422 Fold 0.756198347107438
Chunk 422 Fold 0.6983471074380165
Chunk 422 Fold 0.8801652892561983
Chunk 422 Fold 0.8574380165289256
Chunk 422 Fold 0.7943722943722944
Chunk 422 Fold 0.7316017316017316
Chunk 422 Fold 0.8163265306122449
Chunk 422 Fold 0.6258503401360545
Chunk 423 Fold 0.8768939393939394
Chunk 423 Fold 0.8162878787878788
Chunk 423 Fold 0.846590909090909
Chunk 423 Fold 0.8333333333333333
Chunk 423 Fold 0.8

Chunk 443 Fold 0.9168241965973536
Chunk 443 Fold 0.8771266540642721
Chunk 443 Fold 0.7844990548204158
Chunk 443 Fold 0.8636363636363635
Chunk 443 Fold 0.847107438016529
Chunk 444 Fold 0.8608
Chunk 444 Fold 0.7152000000000001
Chunk 444 Fold 0.8224
Chunk 444 Fold 0.856
Chunk 444 Fold 0.8216666666666667
Chunk 444 Fold 0.8194444444444444
Chunk 444 Fold 0.859375
Chunk 444 Fold 0.8680555555555556
Chunk 444 Fold 0.890625
Chunk 444 Fold 0.8159722222222222
Chunk 444 Fold 0.7881944444444444
Chunk 445 Fold 0.8277591973244147
Chunk 445 Fold 0.8
Chunk 445 Fold 0.8104347826086956
Chunk 445 Fold 0.7826086956521738
Chunk 445 Fold 0.8904347826086956
Chunk 445 Fold 0.8139130434782609
Chunk 445 Fold 0.6730434782608696
Chunk 445 Fold 0.6745454545454546
Chunk 445 Fold 0.7472727272727272
Chunk 445 Fold 0.7563636363636363
Chunk 445 Fold 0.7963636363636364
Chunk 446 Fold 0.7504725897920604
Chunk 446 Fold 0.7466918714555766
Chunk 446 Fold 0.6739130434782608
Chunk 446 Fold 0.7213438735177865
Chunk 446 Fold 0.75

Chunk 467 Fold 0.7430830039525692
Chunk 467 Fold 0.7747035573122529
Chunk 468 Fold 0.8224637681159421
Chunk 468 Fold 0.7580340264650284
Chunk 468 Fold 0.720226843100189
Chunk 468 Fold 0.6521739130434782
Chunk 468 Fold 0.7844990548204158
Chunk 468 Fold 0.7013232514177693
Chunk 468 Fold 0.8166351606805292
Chunk 468 Fold 0.829867674858223
Chunk 468 Fold 0.8071833648393195
Chunk 468 Fold 0.7958412098298677
Chunk 468 Fold 0.890359168241966
Chunk 469 Fold 0.9039855072463768
Chunk 469 Fold 0.7137681159420289
Chunk 469 Fold 0.725378787878788
Chunk 469 Fold 0.7747035573122529
Chunk 469 Fold 0.7213438735177866
Chunk 469 Fold 0.782608695652174
Chunk 469 Fold 0.6324110671936759
Chunk 469 Fold 0.7648221343873518
Chunk 469 Fold 0.8596837944664031
Chunk 469 Fold 0.691699604743083
Chunk 469 Fold 0.5810276679841897
Chunk 470 Fold 0.8066666666666666
Chunk 470 Fold 0.7133333333333334
Chunk 470 Fold 0.7116666666666668
Chunk 470 Fold 0.8483333333333333
Chunk 470 Fold 0.8499999999999999
Chunk 470 Fold 0.743

Chunk 490 Fold 0.8923395445134575
Chunk 490 Fold 0.8874458874458875
Chunk 490 Fold 0.8852813852813852
Chunk 491 Fold 0.782608695652174
Chunk 491 Fold 0.6767485822306237
Chunk 491 Fold 0.7920604914933838
Chunk 491 Fold 0.6185770750988142
Chunk 491 Fold 0.8241106719367589
Chunk 491 Fold 0.6482213438735178
Chunk 491 Fold 0.8596837944664031
Chunk 491 Fold 0.9071146245059288
Chunk 491 Fold 0.8083003952569169
Chunk 491 Fold 0.6897233201581028
Chunk 491 Fold 0.828512396694215
Chunk 492 Fold 0.7916666666666667
Chunk 492 Fold 0.7966666666666667
Chunk 492 Fold 0.8616666666666667
Chunk 492 Fold 0.8266666666666665
Chunk 492 Fold 0.8166666666666667
Chunk 492 Fold 0.8226086956521739
Chunk 492 Fold 0.8260869565217392
Chunk 492 Fold 0.8956521739130434
Chunk 492 Fold 0.7513043478260869
Chunk 492 Fold 0.8208695652173913
Chunk 492 Fold 0.7989130434782609
Chunk 493 Fold 0.8608695652173913
Chunk 493 Fold 0.8643478260869566
Chunk 493 Fold 0.8434782608695652
Chunk 493 Fold 0.7644927536231885
Chunk 493 Fold 0

(array([0.79933646, 0.56128599, 0.49122451, ..., 0.70082502, 1.26528145,
        0.25638205]),
 array([0.91367851, 1.59489841, 0.70418146, ..., 0.68156964, 3.55473555,
        0.07672342]))

In [79]:
# get useful train set from train and test data

# get useful test 
test['target'] = preds
test.loc[test['target'] > 0.985, 'target'] = 1
test.loc[test['target'] < 0.015, 'target'] = 0
usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]

# get useful train 
new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
new_train.loc[oof > 0.985, 'target'] = 1
new_train.loc[oof < 0.015, 'target'] = 0

In [ ]:
oof1, preds1 = instant_model(new_train, test)

In [74]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission1.csv',index=False)